<a href="https://colab.research.google.com/github/mizuresort/competiton_history/blob/main/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install matplotlib-fontja

In [ ]:
# Google Driveのマウント（Colab上で自分のデータにアクセスするための設定です）
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_fontja
import seaborn as sns

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, early_stopping
from sklearn.metrics import accuracy_score



In [ ]:
#データの読み込み
test_df = pd.read_csv('/content/drive/MyDrive/Titanic/data/test.csv')
train_df = pd.read_csv('/content/drive/MyDrive/Titanic/data/train.csv')

In [ ]:
#データの確認
print("==trainデータ==")
train_df.head()
print("==testデータ==")
test_df.head()

==trainデータ==
==testデータ==


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#欠損値があるか確認
train_df.isnull().sum()



,0
PassengerId,0
Perished,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


trainデータ内で欠損値が<br>
Age, Cabin, Embarkedにある。


In [ ]:
test_df.isnull().sum()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


testデータ内でAge,Fare,Cabinに欠損あり
<br>trainデータ内で欠損値が
Age, Cabin, Embarkedにある。

In [ ]:
#(train)欠損値処理 Ageのみ
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)

#(test)欠損値処理
test_df['Age'].fillna(train_df['Age'].median(), inplace=True)

/tmp/ipython-input-360379124.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
/tmp/ipython-input-360379124.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [ ]:
#名前のMrなどを抽出する関数
def extract_prefix(name):
  try:
    return name.split(',')[1].split()[0]
  except:
    return 'Unknown'

train_df['Prefix'] = train_df['Name'].apply(extract_prefix)
test_df['Prefix'] = test_df['Name'].apply(extract_prefix)

In [ ]:
#(train)one-hot encoding
dummy_train_df = pd.get_dummies(
    train_df,
    columns=['Sex', 'Pclass','Prefix' ],
    drop_first=False,
)

print(dummy_train_df.head())

#(test)one-hot encoding
dummy_test_df = pd.get_dummies(
    test_df,
    columns=['Sex', 'Pclass', 'Prefix'],
    drop_first=False,
)

   PassengerId  Perished                                               Name  \
0            1         1                            Braund, Mr. Owen Harris   
1            2         0  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3         0                             Heikkinen, Miss. Laina   
3            4         0       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5         1                           Allen, Mr. William Henry   

    Age  SibSp  Parch            Ticket     Fare Cabin Embarked  ...  \
0  22.0      1      0         A/5 21171   7.2500   NaN        S  ...   
1  38.0      1      0          PC 17599  71.2833   C85        C  ...   
2  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S  ...   
3  35.0      1      0            113803  53.1000  C123        S  ...   
4  35.0      0      0            373450   8.0500   NaN        S  ...   

   Prefix_Master.  Prefix_Miss.  Prefix_Mlle.  Prefix_Mme.  Prefix_Mr.  \
0           False 

In [ ]:
# KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
dummy_train_df['Cluster'] = kmeans.fit_predict(dummy_train_df[features])

# test 側の欠損を補完してから predict
for col in features:
    if dummy_test_df[col].isnull().sum() > 0:
        dummy_test_df[col].fillna(dummy_train_df[col].median(), inplace=True)

dummy_test_df['Cluster'] = kmeans.predict(dummy_test_df[features])


/tmp/ipython-input-1931771968.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dummy_test_df[col].fillna(dummy_train_df[col].median(), inplace=True)


In [ ]:
#目的変数と特徴量に分割
X = dummy_train_df.drop(columns=['Perished', 'Name', 'Ticket', 'Cabin', 'Embarked'])
y = dummy_train_df['Perished']

X_test = dummy_test_df.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'])

for col in X.columns:
    if col not in dummy_test_df.columns:
        dummy_test_df[col] = 0

# 列順も train に揃える
X_test = dummy_test_df[X.columns]

In [ ]:
#訓練データとテストデータに分ける
X_train, X_val, y_train, y_val = train_test_split(
                                X, y, test_size=0.2, random_state=42 )

In [ ]:
#lightgbm
lgbm = lgb.LGBMClassifier(
    objective='binary',
    learning_rate=0.1,
    num_leaves=31,
    random_state=42
)

lgbm.fit(X_train, y_train, eval_set=[(X_val, y_val)],
          callbacks=[early_stopping(stopping_rounds=10)] )

[LightGBM] [Info] Number of positive: 444, number of negative: 268
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 445
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.623596 -> initscore=0.504838
[LightGBM] [Info] Start training from score 0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

LGBMClassifier(objective='binary', random_state=42)

In [ ]:
#評価する
y_pred = lgbm.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {acc:.4f}')

Validation Accuracy: 0.8324


In [ ]:
#テストデータを予測する
test_pred = lgbm.predict(X_test)
dummy_test_df['Perished'] = test_pred

In [ ]:
#CSVに出力
dummy_test_df[['PassengerId', 'Perished']].to_csv('result.csv', index=False)